In [1]:
import csv
import json
import twitter
import pandas as pd
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer
import matplotlib.pyplot as plt
import numpy as np
import math
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

consumer_key = "XlYhjlUyXVCJPPjsIx0dyHrpi"
consumer_secret = "DGA59e7I10JgXTIXwNGqUKVdQcEq6ZZGTczPaE6NmrhfD1i32k"
access_token_key = "1063691900-1skzNu89srqH1rJWEb2KdcDFHUuwlm4QzuMlkbT"
access_token_secret = "0kBGxT6rxfNuL2vpqYPRItiyEyEQmTS2cTuKdDwfVXgSb"


In [2]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [6]:
quebecMedia = ["@tvanouvelles","@JdeMontreal","@LP_LaPresse","@RadioCanadaInfo","@JdeQuebec","@vicequebec",
"@infomantv","@salutbonjour","@CBCMontreal","@TVASports","@RDSca","@icimontreal","@meteomedia","@CTVMontreal",
"@Global_Montreal","@telequebec","@LaTribune","@_URBANIA","@HuffPostQuebec","@LeDevoir","@mtlgazette","@iciquebec",
"@cyblesoleil","@Lactualite","@coupdepouce_mag","@la_lesaffaires","@journalmetro","icircpremiere"]

francemedia = ["@Le_Figaro","@lemondefr","@canardenchaine","@Qofficiel", "@20hFrance2", "@TF1LeJT", "@LCI", "@BFMTV", 
              "@CNEWS", "@libe", "@lequipe", "@le_Parisien", "@20Minutes", "@lobs" ,"@LesEchos","@TV5MONDE", 
              "@Le_Soir3", "@ARTEjournal", "@LaTribune", "@franceinfo", "@FRANCE24","@rtlinfo","@Europe1",
              "@RMCinfo", "@franceinter","@francebleu","@Charlie_Hebdo_"]



columns_to_keep = ["created_at","display_text_range","entities","favorite_count","full_text","retweet_count","sentiment","searchTerm"]

In [7]:
len(francemedia)

27

In [3]:
t = twitter.Api(consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token_key=access_token_key,
    access_token_secret=access_token_secret,
    tweet_mode="extended"
    )

In [15]:
def tweetsSearch(searchTerm="",lang=None,since="2019-03-10",until="2019-03-18"):
    tweets = t.GetSearch(term = searchTerm,count = 100,return_json = True,lang=lang,since=since, until=until, result_type ="mixed")
    return tweets

In [19]:
def weekScrap(searchTerm):
    date = "2019-03-1"
    for i in range(1,8):
        tweets = tweetsSearch(searchTerm=searchTerm,lang='fr',since=date+str(i),until=date+str(i+1))
        if(i==1):
            pd_tweets = pd.DataFrame(tweets["statuses"])
        else:
            pd_tweets = pd.concat([pd_tweets,pd.DataFrame(tweets["statuses"])])
    return pd_tweets

In [70]:
def getRTStatusId(row):
    try:
        if(math.isnan(row["retweeted_status"])):
            return row["id"]
    except:
        return row["retweeted_status"]["id"]

def replaceRTFullText(row):
    try:
        if(math.isnan(row["retweeted_status"])):
            return row["full_text"]
    except:
        return row['retweeted_status']["full_text"]

    
def rmDuplicatedRT(df):
    df['original_status_id'] = df.apply(lambda row: getRTStatusId(row),axis=1)    
    df['full_text'] = df.apply(lambda row: replaceRTFullText(row),axis=1)
    return df.drop_duplicates(subset="original_status_id")

In [391]:
print(len(pd_tweets["full_text"].values))
pd_tweets = rmDuplicatedRT(pd_tweets)
print(len(pd_tweets["full_text"].values))

425
135


In [7]:
def sentiment_score(sentence):
    fr_blob = TextBlob(sentence)
    try:
        #translate sentence to english
        sentence_eng = fr_blob.translate(to='eng')
    except:
        sentence_eng = sentence
    #compute the polarity score -1 = negative, 1 = positive (continuous values)
    snt = analyser.polarity_scores(sentence_eng)
    return snt["compound"]

In [8]:
def cleanScrap(searchterm):
    #Scrap tweets
    pd_tweets = weekScrap(searchterm)
    
    #Keep one copy of retweeted tweets
    pd_tweets = rmDuplicatedRT(pd_tweets)
    
    #compute the sentiment of tweets
    pd_tweets["sentiment"] = pd_tweets.apply(lambda row: sentiment_score(row["full_text"]),axis=1)
    
    pd_tweets["searchTerm"] = searchterm
    
    return pd_tweets

def fullScrap(searchterm_list):
    init = False
    
    for term in searchterm_list:
        if(not init):
            df_all_tweet = cleanScrap(term)
            print(len(df_all_tweet["full_text"].values))
            init=True
        else:
            df_all_tweet = pd.concat([df_all_tweet,cleanScrap(term)])
            print(len(df_all_tweet["full_text"].values))

    return df_all_tweet

def saveToCSV(df,filePath, sep="|"):
    df[columns_to_keep].to_csv(filePath,sep="|", encoding='utf-8')

In [72]:
pd_tweets = cleanScrap("@LeMonde")
pd_tweets.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,truncated,user,original_status_id,sentiment,searchTerm
0,None,None,Mon Mar 11 23:43:41 +0000 2019,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,C’est paru en Livre de Poche! 30 femmes racont...,None,...,NaN,65,False,{'created_at': 'Mon Mar 11 14:07:10 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 892083156509216768, 'id_str': '89208315...",1105107901889302529,0.6892,@LeMonde
10,None,None,Mon Mar 11 21:31:09 +0000 2019,"[0, 280]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,3,False,Il n’y a pas de « normes » post fukushima mais...,None,...,NaN,2,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4904726237, 'id_str': '4904726237', 'na...",1105219634217865219,-0.8807,@LeMonde
12,None,None,Mon Mar 11 21:17:59 +0000 2019,"[0, 98]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,@lemonde_pol \n@lemonde \nMourrir dans l'Obscu...,None,...,1105159080845422592,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 966716686182375425, 'id_str': '96671668...",1105216319023124483,-0.7096,@LeMonde
28,None,None,Mon Mar 11 16:39:21 +0000 2019,"[0, 140]","{'hashtags': [{'text': 'AutreAffaireDuSiècle',...",NaN,0,False,#AutreAffaireDuSiècle: les multinationales (et...,None,...,NaN,15,False,{'created_at': 'Mon Mar 11 08:56:12 +0000 2019...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 187096235, 'id_str': '187096235', 'name...",1105029644003274752,0.5859,@LeMonde
31,None,None,Mon Mar 11 16:29:14 +0000 2019,"[0, 162]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,"le calendrier initial n'était pas tenable"", a ...",None,...,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,"{'id': 1460092549, 'id_str': '1460092549', 'na...",1105143656468660225,-0.4588,@LeMonde


In [440]:
df_quebec = fullScrap(quebecMedia)
saveToCSV(df_quebec,"QuebecMedia")

In [22]:
df_monde = fullScrap(["@LeMonde"])

108


In [75]:
data = [go.Histogram(x=pd_tweets["sentiment"])]
iplot(data)

In [70]:
print(len(pd_tweets))
pd_tweets.columns

100


Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'favorite_count', 'favorited',
       'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
       'lang', 'metadata', 'place', 'possibly_sensitive', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'retweet_count',
       'retweeted', 'retweeted_status', 'source', 'truncated', 'user'],
      dtype='object')

In [3]:
from nbconvert import HTMLExporter
import codecs
import nbformat
exporter = HTMLExporter()
output_notebook = nbformat.read('TwitterScrapping.ipynb', as_version=4)
output, resources = exporter.from_notebook_node(output_notebook)
codecs.open('TwitterScrapping.html', 'w', encoding='utf-8').write(output)